# TextMining Final Project (BioBert)
Lucas de Wolff (s3672980) and Ruben Ahrens (s3677532)

January 2024


In [1]:
import datasets
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from huggingface_hub import interpreter_login
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from helper import write_tex_table

In [2]:
names = ['O','B-adr', 'B-disease', 'B-drug', 'B-symptom', 'B-finding','I-adr', 'I-disease', 'I-drug', 'I-symptom', 'I-finding']

In [3]:
train, valid, test = 'train.ann', 'valid.ann', 'test.ann'
path = '../../cadec/processed/'

raw_datasets = {}
for file, name in zip([train, valid, test], ['train', 'validation', 'test']):
    id = 0
    raw_datasets[name] = {'id': [], 'tokens': [], 'ner_tags': []}
    with open(f'{path}/{file}', 'r') as f:
        tokens, ner_tags = [], []
        for line in f:
            try:
                token, ner_tag = line.split()
                tokens.append(token)
                ner_tags.append(names.index(ner_tag))
            except:
                raw_datasets[name]['id'].append([id for _ in range(len(tokens))])
                raw_datasets[name]['tokens'].append(tokens)
                raw_datasets[name]['ner_tags'].append(ner_tags)
                id += 1
                tokens, ner_tags = [], []
    raw_datasets[name] = datasets.Dataset.from_dict(raw_datasets[name])

raw_datasets = datasets.DatasetDict(raw_datasets)
display(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 988
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 185
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 132
    })
})

# Processing the data

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

# compare total number of tokens before and after tokenization. which is not the number of rows, but the number of tokens
train_set = set([word  for sentence  in  raw_datasets['train']['tokens'] for word in sentence ])
eval_set = set([word  for  sentence  in raw_datasets['validation']['tokens'] for word in sentence ])
test_set = set([word  for  sentence  in raw_datasets['test']['tokens'] for word in sentence ])

print(len(train_set), len(eval_set), len(test_set))

# Print unique tokens of each dataset
train_set_unique = train_set - eval_set - test_set
eval_set_unique = eval_set - train_set - test_set
test_set_unique = test_set - train_set - eval_set

print(len(train_set_unique), len(eval_set_unique), len(test_set_unique))

## Data collation

In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Metrics

In [12]:
metric = evaluate.load("seqeval")

In [13]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [14]:
def compute_all_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return all_metrics

## Defining the model

In [15]:
id2label = {i: label for i, label in enumerate(names)}
label2id = {v: k for k, v in id2label.items()}

In [16]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##  Hyperparameter Optimization

In [17]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Token is valid (permission: read).
Your token has been saved to C:\Users\lcde

In [19]:
training_args = TrainingArguments(
    "biobert-cadec-ner-optuna",
    evaluation_strategy="epoch",
    logging_strategy="no",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    # push_to_hub=True,
    eval_accumulation_steps=1,
)

def model_init(): 
    return AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "adam_beta1": trial.suggest_float("beta_1", 0.9, 0.999),
        "adam_beta2": trial.suggest_float("beta_2", 0.9, 0.999),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

best_run = trainer.hyperparameter_search(
    direction="minimize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=25,
)

# import os
# import torch
# import gc
# from transformers.trainer_utils import HPSearchBackend, default_hp_space

# def run_hp_search_optuna(trainer, n_trials, direction, **kwargs):
#     import optuna
    
#     def _objective(trial, checkpoint_dir=None):
#         checkpoint = None
#         if checkpoint_dir:
#             for subdir in os.listdir(checkpoint_dir):
#                 if subdir.startswith(PREFIX_CHECKPOINT_DIR):
#                     checkpoint = os.path.join(checkpoint_dir, subdir)
#         #################
#         ## UPDATES START
#         #################
#         if not checkpoint:
#             # free GPU memory
#             del trainer.model
#             gc.collect()
#             torch.cuda.empty_cache()
#         trainer.objective = None
#         trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
#         # If there hasn't been any evaluation during the training loop.
#         if getattr(trainer, "objective", None) is None:
#             metrics = trainer.evaluate()
#             trainer.objective = trainer.compute_objective(metrics)
#         return trainer.objective

#     timeout = kwargs.pop("timeout", None)
#     n_jobs = kwargs.pop("n_jobs", 1)
#     study = optuna.create_study(direction=direction, **kwargs)
#     study.optimize(_objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)
#     best_trial = study.best_trial
#     return BestRun(str(best_trial.number), best_trial.value, best_trial.params)

# def hyperparameter_search(trainer, compute_objective, n_trials, direction, **kwargs):
#     trainer.hp_search_backend = HPSearchBackend.OPTUNA
#     trainer.hp_space = optuna_hp_space
#     trainer.hp_name = None
#     trainer.compute_objective = compute_objective
#     best_run = run_hp_search_optuna(trainer, n_trials, direction, **kwargs)
#     self.hp_search_backend = None
#     return best_run

# best_run = hyperparameter_search(
#     trainer=trainer,
#     compute_objective=compute_metrics,
#     n_trials=25,
#     direction="minimize",
# )

C:\Users\lcdew\AppData\Roaming\Python\Python38\site-packages\transformers\trainer.py:351: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2024-01-04 12:08:12,397] A new study created in memory with name: no-name-123fd644-64d5-4f2c-9011-e14438343883
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[W 2024-01-04 12:08:13,581] Trial 0 failed with parameters: {'learning_rate': 4.606075704900877e-06, 'per_device_train_batch_size': 32, 'beta_1': 0.9598341512018522, 'beta_2': 0.966011782157629, 'weight_decay': 0.07067093535594578, 'adam_epsilon': 2.815709670111468e-08}

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 6.00 GiB total capacity; 11.54 GiB already allocated; 0 bytes free; 12.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [52]:
with open('Results/biobert/best_run_biobert_cadec_ner_optuna.txt', 'wb') as f:
    for key, value in best_run.hyperparameters.items():
        f.write(f'{key}: {value}\n'.encode('utf-8'))

## Retrain model with optimized hyperparameters

In [21]:
training_args = TrainingArguments(
    "biobert-cadec-ner",
    evaluation_strategy="epoch",
    logging_strategy="no",
    save_strategy="no",
    num_train_epochs=50,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

# Load from text file
# with open('best_run_biobert_cadec_ner_optuna.txt', 'r') as f:
#     for line in f:
#         n, v = line.split(': ')
#         setattr(trainer.args, n, v)
    
trainer.train()
trainer.push_to_hub(commit_message="Training complete")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 11.95 GiB already allocated; 0 bytes free; 12.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Evaluating Model

In [54]:
predictions = trainer.predict(tokenized_datasets["test"])
eval_preds = predictions.predictions, predictions.label_ids

100%|██████████| 17/17 [00:12<00:00,  1.34it/s]


In [55]:
all_metrics = compute_all_metrics(eval_preds)
logits, labels = eval_preds
predictions = np.argmax(logits, axis=-1)

# Remove ignored index (special tokens) and convert to labels
true_labels = [[names[l] for l in label if l != -100] for label in labels]
true_predictions = [
    [names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

from seqeval.metrics import classification_report
print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

         adr       0.00      0.00      0.00       800
     disease       1.00      0.01      0.01       171
        drug       0.00      0.00      0.00       217
     finding       0.00      0.00      0.00        60
     symptom       0.00      0.00      0.00        80

   micro avg       1.00      0.00      0.00      1328
   macro avg       0.20      0.00      0.00      1328
weighted avg       0.13      0.00      0.00      1328



In [ ]:
# flatten the labels and predictions
true_labels_flat = [item for sublist in true_labels for item in sublist]
true_predictions_flat = [item for sublist in true_predictions for item in sublist]

# compute precision, recall, and f1 per entity type
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, true_predictions_flat, average=None, labels=names[1:])

# count number of entities per type
from collections import Counter
count = Counter(true_labels_flat)

# format into pandas table and round to 2 decimals
import pandas as pd
df = pd.DataFrame({'precision': precision, 'recall': recall, 'f1': f1, 'number': count}, index=names[1:])
df = df.round(2)
display(df)
write_tex_table(df.to_dict('index'), 'Results/BioBert/biobert-finetuned-evaluationmetrics.txt')